In [1]:
# #Download and extract data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
tar: Error opening archive: Failed to open '20021010_easy_ham.tar.bz2'
tar: Error opening archive: Failed to open '20021010_hard_ham.tar.bz2'
tar: Error opening archive: Failed to open '20021010_spam.tar.bz2'


In [2]:
!ls -lah


total 1232
drwxr-xr-x  8 apoorva  staff   256B Dec  6 19:07 .
drwx------+ 7 apoorva  staff   224B Dec  5 20:28 ..
-rw-r--r--@ 1 apoorva  staff   6.0K Dec  5 21:26 .DS_Store
drwxr-xr-x  5 apoorva  staff   160B Dec  5 21:56 .ipynb_checkpoints
-rw-r--r--  1 apoorva  staff   262K Dec  5 20:27 Classification of Iris dataset using supervised and unsupervised learning.ipynb
-rw-r--r--  1 apoorva  staff   311K Dec  5 20:29 Predicting the sales of villas in Landvetter using Linear Regression.ipynb
-rw-r--r--  1 apoorva  staff    24K Dec  6 19:07 Spam and ham mail classification.ipynb
-rw-r--r--@ 1 apoorva  staff   1.5K Dec  5 20:23 data_assignment2.csv


In [3]:
# Importing necessary packages:

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#pre-processing code here

import pandas as pd
from sklearn.model_selection import train_test_split
import tarfile 

#Extracting emails from zip file, storing them in a dataframe
def extract_emails(fnames):
    """ Extract the zipped emails and load them into a pandas df.
    Args:
        fname (str): the files with tar.bz2 extension
    Returns:
        pandas df: a pandas dataframe of emails
    """
#     print(fnames)
    rows = []
    for fname in fnames:
#         print(fname)
        tfile = tarfile.open(fname, 'r:bz2')
        for member in tfile.getmembers():
            if 'ham' in member.name:
                f = tfile.extractfile(member)
                if f is not None:
                    row = f.read()
                    rows.append({'message': row.decode('latin-1'), 'class': 0, 'category':'ham', })
            if 'spam' in member.name:
                f = tfile.extractfile(member)
                if f is not None:
                    row = f.read()
                    rows.append({'message': row.decode('latin-1'), 'class': 1, 'category':'spam'})
        tfile.close()
    return pd.DataFrame(rows)

easy_ham = extract_emails(['20021010_easy_ham.tar.bz2'])
hard_ham = extract_emails(['20021010_hard_ham.tar.bz2'])
ham = extract_emails(['20021010_easy_ham.tar.bz2','20021010_hard_ham.tar.bz2'])
spam = extract_emails(['20021010_spam.tar.bz2'])

#Printing to see the message content of a sample email:
# print(df_easy_ham['message'][0])

print("Easy ham emails count: {0}".format(easy_ham.shape[0]))
print("Hard ham emails count: {0}".format(hard_ham.shape[0]))
print("Ham emails count: {0}".format(ham.shape[0]))
print("Spam emails count: {0}".format(spam.shape[0]))

FileNotFoundError: [Errno 2] No such file or directory: '20021010_easy_ham.tar.bz2'

In [ ]:
ham.head()
# ham.groupby('class').describe()

In [ ]:
#Code here

# X_hamtrain, X_hamtest, y_hamtrain,  y_hamtest  = train_test_split(ham['message'], ham['class'],  test_size=0.20, random_state=42)
hamtrain,  hamtest  = train_test_split(ham,  test_size=0.20, random_state=0)
spamtrain, spamtest = train_test_split(spam, test_size=0.20, random_state=0)

print("Ham train emails count: {0}".format(hamtrain.shape[0]))
print("Ham test emails count: {0}".format(hamtest.shape[0]))
print("Spam train emails count: {0}".format(spamtrain.shape[0]))
print("Spam test emails count: {0}".format(spamtest.shape[0]))

In [ ]:
# Concatenating ham and spam training emails as one database - train_mails
train_mails = pd.concat([hamtrain,spamtrain])
# Concatenating ham and spam test emails as one database - test_mails
test_mails = pd.concat([hamtest,spamtest])

In [ ]:
# Function email_classifier defined for email classification:
def email_classifier(model,X_train,X_test,y_train,y_test):
    """
    INPUT Parameters:
    model    : Input model with CountVectorizer and Classifier defined in a pipeline
    X_train  : Training emails message 
    X_test   : Test emails message
    y_train  : Train emails label, specifying ham or spam
    y_test   : Test emails label, specifying ham or spam
    OUPUT Parameters:
    accuracy : score calculated by the model with predicted and actual labels
    cm       : Confusion Matrix
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test,y_test)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    fnr = fn/(fn+tp)
    return accuracy, cm, tpr, fnr

In [ ]:
# Model 1 with CountVectorizer and Multinomial Naive Bayes in a pipeline
model_mnb = Pipeline([('vectorizer', CountVectorizer()),('mnb', MultinomialNB())])
# Model 2 with CountVectorizer and Bernouli Naive Bayes in a pipeline
model_bnb = Pipeline([('vectorizer', CountVectorizer()),('bnb', BernoulliNB())])

In [ ]:
accuracy_mnb, cm_mnb, tpr_mnb, fnr_mnb = email_classifier(model_mnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])
accuracy_bnb, cm_bnb, tpr_bnb, fnr_bnb = email_classifier(model_bnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])

In [ ]:
print("=============== Multinomial Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_mnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_mnb))
print("False Negative Rate : {0:.3f}".format(fnr_mnb))
print('\n')
print("=============== Bernoulli Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_bnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_bnb))
print("False Negative Rate : {0:.3f}".format(fnr_bnb))

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_mnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Blues');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Multinomial Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy_mnb*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_bnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Greens');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Bernoulli Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy_bnb*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
#Code to report results here
easyhamtrain,  easyhamtest  = train_test_split(easy_ham,  test_size=0.20, random_state=0)
hardhamtrain,  hardhamtest  = train_test_split(hard_ham,  test_size=0.20, random_state=0)

print("Easy Ham train emails count: {0}".format(easyhamtrain.shape[0]))
print("Easy Ham test emails count : {0}".format(easyhamtest.shape[0]))
print("Hard Ham train emails count: {0}".format(hardhamtrain.shape[0]))
print("Hard Ham test emails count : {0}".format(hardhamtest.shape[0]))
print("Spam train emails count    : {0}".format(spamtrain.shape[0]))
print("Spam test emails count     : {0}".format(spamtest.shape[0]))

In [ ]:
# Concatenating easy ham and spam training emails as one database - train_mails
train_mails = pd.concat([easyhamtrain,spamtrain])
# Concatenating easy ham and spam test emails as one database - test_mails
test_mails = pd.concat([easyhamtest,spamtest])

print("Total train emails count: {0}".format(train_mails.shape[0]))
print("Total test emails count : {0}".format(test_mails.shape[0]))

In [ ]:
#Multinominal Naive Bayes
accuracy_mnb, cm_mnb, tpr_mnb, fnr_mnb = email_classifier(model_mnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])
#Bernouli Naive Bayes
accuracy_bnb, cm_bnb, tpr_bnb, fnr_bnb = email_classifier(model_bnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])

print("Spam vs Easy ham:")
print("=============== Multinomial Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_mnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_mnb))
print("False Negative Rate : {0:.3f}".format(fnr_mnb))
print('\n')
print("=============== Bernoulli Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_bnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_bnb))
print("False Negative Rate : {0:.3f}".format(fnr_bnb))

In [ ]:
#Multinomial Naive bayes confusion matrix
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_mnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Blues');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Multinomial Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy_mnb*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
#Bernouli Naive bayes confusion matrix
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_bnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Greens');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Bernoulli Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy_bnb*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
# Concatenating hard ham and spam training emails as one database - train_mails
train_mails = pd.concat([hardhamtrain,spamtrain])
# Concatenating hard ham and spam test emails as one database - test_mails
test_mails = pd.concat([hardhamtest,spamtest])

print("Total train emails count: {0}".format(train_mails.shape[0]))
print("Total test emails count : {0}".format(test_mails.shape[0]))

In [ ]:
#Multinominal Naive Bayes
accuracy_mnb, cm_mnb, tpr_mnb, fnr_mnb  = email_classifier(model_mnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])
#Bernouli Naive Bayes
accuracy_bnb, cm_bnb, tpr_bnb, fnr_bnb = email_classifier(model_bnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])

print("Spam vs Hard ham:")
print("=============== Multinomial Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_mnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_mnb))
print("False Negative Rate : {0:.3f}".format(fnr_mnb))
print('\n')
print("=============== Bernoulli Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_bnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_bnb))
print("False Negative Rate : {0:.3f}".format(fnr_bnb))

In [ ]:
#Multinomial Naive bayes confusion matrix
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_mnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Blues');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Multinomial Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy_mnb*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
#Bernouli Naive bayes confusion matrix
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_bnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Greens');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Bernoulli Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy_bnb*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter 
import collections


def filter_words(emails, uncommon_words_limit, common_words_limit):
    words_occurance = dict()
    for email in emails:
        words = word_tokenize(email)
        counter = Counter(words)
        ten_repeated = counter.most_common(10)
        
        for (word,count) in ten_repeated:
            if word.isalpha():
                if word in words_occurance:
                    words_occurance[word] += count
                else:
                    words_occurance[word] = count
    
    temp = collections.OrderedDict(sorted(words_occurance.items(), key=lambda kv: kv[1], reverse=True))
    filtered = {key:value for key, value in temp.items() if value < uncommon_words_limit or value > common_words_limit}
    return filtered


In [ ]:
print("Common and Uncommon words in ham emails:",filter_words(ham['message'],20,1000))

In [ ]:
print("Common and Uncommon words in spam emails:",filter_words(spam['message'],20,1000))

In [ ]:


# Model 1 with CountVectorizer and Multinomial Naive Bayes in a pipeline
model_mnb = Pipeline([('vectorizer', CountVectorizer(min_df=10, max_df=1000)),('mnb', MultinomialNB())])
# Model 2 with CountVectorizer and Bernouli Naive Bayes in a pipeline
model_bnb = Pipeline([('vectorizer', CountVectorizer(min_df=10, max_df=1000)),('bnb', BernoulliNB())])

# Concatenating ham and spam training emails as one database - train_mails
train_mails = pd.concat([hamtrain,spamtrain])
# Concatenating ham and spam test emails as one database - test_mails
test_mails = pd.concat([hamtest,spamtest])


In [ ]:
print("=============== Multinomial Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_mnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_mnb))
print("False Negative Rate : {0:.3f}".format(fnr_mnb))
print('\n')
print("=============== Bernoulli Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_bnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_bnb))
print("False Negative Rate : {0:.3f}".format(fnr_bnb))

In [ ]:
# Function email_classifier defined for email classification:
def email_classifier_mod(model,X_train,X_test,y_train,y_test):
    """
    INPUT Parameters:
    model    : Input model with CountVectorizer and Classifier defined in a pipeline
    X_train  : Training emails message 
    X_test   : Test emails message
    y_train  : Train emails label, specifying ham or spam
    y_test   : Test emails label, specifying ham or spam
    OUPUT Parameters:
    accuracy : score calculated by the model with predicted and actual labels
    cm       : Confusion Matrix
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test,y_test)
    cm = confusion_matrix(y_test, y_pred)
#     print(cm)
    tn, fp, fn, tp = cm.ravel()
#     tpr = tp/(tp+fn)
#     fnr = fn/(fn+tp)
    return accuracy, cm

In [ ]:
accuracy1, cm_mnb = email_classifier_mod(model_mnb, spamtrain['message'], hamtest['message'], spamtrain['class'], hamtest['class'])
accuracy2, cm_bnb = email_classifier_mod(model_bnb, spamtrain['message'], hamtest['message'], spamtrain['class'], hamtest['class'])

In [ ]:
#Multinomial Naive bayes confusion matrix
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_mnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Blues');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Multinomial Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy1*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
#Bernouli Naive bayes confusion matrix
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(cm_bnb, annot=True,fmt=".3f", linewidths=0.5,square = True, cmap = 'Greens');
names = np.array(['Ham','Spam'])
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
ax.set_xticklabels(names)
ax.set_yticklabels(names)
title = ('Confusion Matrix for Bernoulli Naive Bayes classifier - Accuracy : {0:.3f}%'.format(accuracy2*100))
plt.title(title, size = 10);
plt.show()

In [ ]:
#fit_prior is set to false, ignoring previous state probabilities

# Model 1 with CountVectorizer and Multinomial Naive Bayes in a pipeline
model_mnb = Pipeline([('vectorizer', CountVectorizer(min_df=10, max_df=1000)),('mnb', MultinomialNB(fit_prior=False))])
# Model 2 with CountVectorizer and Bernouli Naive Bayes in a pipeline
model_bnb = Pipeline([('vectorizer', CountVectorizer(min_df=10, max_df=1000)),('bnb', BernoulliNB(fit_prior=False))])

# Concatenating ham and spam training emails as one database - train_mails
train_mails = pd.concat([hamtrain,spamtrain])
# Concatenating ham and spam test emails as one database - test_mails
test_mails = pd.concat([hamtest,spamtest])



In [ ]:
accuracy_mnb, cm_mnb, tpr_mnb, fnr_mnb = email_classifier(model_mnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])
accuracy_bnb, cm_bnb, tpr_bnb, fnr_bnb = email_classifier(model_bnb, train_mails['message'], test_mails['message'], train_mails['class'], test_mails['class'])

In [ ]:
print("=============== Multinomial Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_mnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_mnb))
print("False Negative Rate : {0:.3f}".format(fnr_mnb))
print('\n')
print("=============== Bernoulli Naive Bayes classifier ===============")
print("Accuracy            : {0:.3f}%".format(accuracy_bnb*100))
print("True Positive Rate  : {0:.3f}".format(tpr_bnb))
print("False Negative Rate : {0:.3f}".format(fnr_bnb))